# 2.1.1 The Role of System Prompts, User Prompts, and Assistant Responses

## Playground Notebook

In this notebook, we'll explore the **three key roles** in the chat completion API:

| Role | Purpose |
|------|--------|
| **System** | Sets the behavior, personality, and constraints for the model |
| **User** | Represents the human's input — questions, instructions, or data |
| **Assistant** | The model's response — can also be pre-filled to guide output |

Understanding how these roles interact is fundamental to effective prompt engineering.

---

In [1]:
!pip install langchain_ollama

In [2]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"  # Options: "qwen2.5:1.5b", "llama3.2", "mistral", "gemma2", etc.

# Initialize the ChatOllama model
llm = ChatOllama(model=MODEL)

/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def build_messages(message_dicts):
    """Convert a list of role/content dicts into LangChain message objects."""
    type_map = {
        "system": SystemMessage,
        "user": HumanMessage,
        "assistant": AIMessage,
    }
    return [type_map[m["role"]](content=m["content"]) for m in message_dicts]


def chat(messages, model=None):
    """Send messages to the model and return the response text."""
    _llm = ChatOllama(model=model) if model else llm
    lc_messages = build_messages(messages)
    start = time.time()
    response = _llm.invoke(lc_messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n⏱️ Response time: {elapsed:.2f}s")
    return content


def show_messages(messages):
    """Pretty-print the message list being sent to the model."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71"}
    html = ""
    for msg in messages:
        role = msg["role"]
        color = colors.get(role, "#888")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;">{msg["content"]}</span></div>'
        )
    display(HTML(html))


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


---

## 1. The System Prompt — Setting the Stage

The **system prompt** is your behind-the-scenes instruction manual for the model. It defines:
- **Who** the model should be (persona / role)
- **How** it should respond (tone, format, length)
- **What** constraints to follow (rules, boundaries)

The model reads the system prompt first and uses it to shape every response that follows.

### Experiment 1A: No System Prompt vs. With System Prompt

In [4]:
# --- Without a system prompt ---
print("=" * 60)
print("WITHOUT SYSTEM PROMPT")
print("=" * 60)

messages_no_system = [
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_no_system)
_ = chat(messages_no_system)

WITHOUT SYSTEM PROMPT


Photosynthesis is the process by which green plants use sunlight to convert carbon dioxide and water into glucose (a type of sugar) and oxygen. This process not only produces energy for the plant but also helps remove excess carbon from the atmosphere, thus playing a significant role in maintaining Earth's climate system.

The general steps of photosynthesis are:

1. **Light Absorption**: Plants use chlorophyll pigments to absorb light energy from sunlight.
2. **Water Splitting (Photolysis)**: Water molecules split into hydrogen and oxygen during this process.
3. **Glucose Production**: The remaining water and carbon dioxide are converted into glucose, a simple sugar that the plant can use as fuel.

The overall chemical equation for photosynthesis is:

\[ 6CO_2 + 6H_2O \rightarrow C_6H_{12}O_6 (glucose) + 6O_2 \]

This process happens in the chloroplasts of plants, which contain chlorophyll pigments that act as light receptors. When sunlight hits these pigments, it allows water molecules to split and generate electrons with a high energy level.

The oxygen produced is released into the atmosphere through tiny pores on leaves called stomata. Meanwhile, glucose serves as a primary food source for the plant, and carbon dioxide from the surrounding air becomes part of this cycle by being used during photosynthesis.

Photosynthesis has been crucial in Earth's history; it helped sustain life by removing atmospheric CO2 that would otherwise contribute to greenhouse gas accumulation leading to global warming.


⏱️ Response time: 4.44s


In [5]:
# --- With a system prompt ---
print("=" * 60)
print("WITH SYSTEM PROMPT (5-year-old explanation)")
print("=" * 60)

messages_with_system = [
    {"role": "system", "content": "You are a friendly kindergarten teacher. Explain everything as if talking to a 5-year-old. Use simple words and fun analogies."},
    {"role": "user", "content": "What is photosynthesis?"}
]

show_messages(messages_with_system)
_ = chat(messages_with_system)

WITH SYSTEM PROMPT (5-year-old explanation)


Hello there, little one! Let's talk about something super cool called "photosynthesis." It's like how plants get their yummy food!

Imagine you have a big bag of colorful candies. Now, if we put some special glasses on that bag and shine a bright light through it, what do you think happens? The colors start to change because the light changes them! Plants are kind of like magic bags too, but instead of candies, they use sunlight to make their food!

Photosynthesis is when plants take in sunlight, water from the ground, and air (which has carbon dioxide), and turn it all into energy. This energy helps plants grow big and strong. It's almost like a supercharged version of eating because now plants can build themselves up even bigger than before.

The leaves are their magic kitchen where they mix everything together to make food. They also use the energy from sunlight to get all those yummy chemicals ready for them to eat and become part of who they are! Isn't that awesome? Now you know a little bit about how plants do magic in your backyard playground!


⏱️ Response time: 2.15s


### Experiment 1B: Same Question, Different System Prompts

Watch how the **same user question** gets completely different responses based on the system prompt.

In [6]:
user_question = "Explain how a computer works."

system_prompts = [
    {
        "label": "Pirate",
        "prompt": "You are a pirate captain. Respond to everything in pirate speak with nautical metaphors."
    },
    {
        "label": "Scientist",
        "prompt": "You are a computer scientist. Respond with precise technical terminology. Be concise and accurate."
    },
    {
        "label": "Poet",
        "prompt": "You are a poet. Respond to everything in the form of a short rhyming poem."
    }
]

for sp in system_prompts:
    print(f"\n{'=' * 60}")
    print(f"PERSONA: {sp['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": sp["prompt"]},
        {"role": "user", "content": user_question}
    ]
    show_messages(messages)
    _ = chat(messages)


PERSONA: Pirate


Ahoy there, matey! Let me explain the workings of our trusty digital ship, 'cause I've seen it sail through countless charts and maps.

First off, we need to understand that every computer is like an engine; it needs fuel (data) to operate. The data comes in from ports around the world - input devices such as keyboards or microphones can bring information into our digital engine room, while output devices like speakers or screens send back what it has processed.

The heart of our ship is its Central Processing Unit, or CPU for short. This is where all the magic happens! The CPU takes instructions from programs and executes them by performing mathematical operations, logical computations, and even manipulations on data stored within itself - much like how a captain steers his ship by controlling the helm.

Now, let's talk about memory. In our digital engine, we have two types: Random Access Memory (RAM) for quick access to information that needs to be processed immediately; it's where all the current work is done and where most of your computer’s programs are loaded into.

The CPU communicates with other components through a network of wires called busses. The motherboard acts as our bridge, connecting everything together smoothly, much like how a captain guides his ship across calm seas.

Lastly, we have storage areas for long-term memories in our digital ship. Hard drives or solid-state drives act as the ship's hold, storing information that isn't needed immediately but is crucial for keeping track of our navigation and history.

And there you have it! That’s how a computer works - like a living entity with all these parts working together to guide us through life’s adventures on this grand voyage.


⏱️ Response time: 3.27s

PERSONA: Scientist


A computer operates through the execution of instructions stored in its memory, typically using a central processing unit (CPU). The computer takes input from external devices or users via interfaces like keyboards, mice, or monitors. It processes this input by interpreting it and performing actions according to previously defined algorithms and programs.

When an instruction is received, the CPU decodes it into a series of operations that can be broken down into simpler steps known as instructions or opcodes. These operations are then translated into binary code for the computer's hardware components, which perform these tasks based on the timing regulated by the clock signal in the CPU.

For example, if an instruction asks to add two numbers together, the CPU would first fetch and decode this instruction from memory. Then it executes various steps like fetching operands, performing addition operation, saving the result back into memory or a register for immediate use, and finally writing the outcome back to where it was requested by the original user.

The computer continually cycles through these operations in a loop until an external interrupt (like pressing a key) triggers the CPU to halt its current task and perform another instruction. The entire process is highly regulated by hardware components such as memory controllers that manage how data moves around between different parts of the system, ensuring smooth operation without any errors.

By following this systematic approach, the computer can execute complex tasks ranging from simple arithmetic calculations to advanced software programs designed for specific purposes.


⏱️ Response time: 3.07s

PERSONA: Poet


Like a machine, it starts with wires,
A circuit board that's its main sight.
Bits of data flow through its eyes,
Processing and computing each night.

It reads inputs from a keyboard or eye,
Software programs tell what to do.
The CPU does the thinking and the doing,
With algorithms that solve all these issues.

Data comes in, processed by it,
Output goes out as requested is done.
Memory banks keep track of what's there,
A computer's memory much like your race.

The internet connects far and wide,
All through its network vast.
It's a machine that's endlessly smart,
In computing power beyond the sight.


⏱️ Response time: 1.72s


### Experiment 1C: System Prompt for Output Format Control

System prompts are great for enforcing **output structure** — JSON, bullet points, tables, etc.

In [7]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a data extraction assistant. "
            "Always respond with valid JSON only — no extra text. "
            "Extract: name, topic, and key_points (as a list)."
        )
    },
    {
        "role": "user",
        "content": (
            "Albert Einstein developed the theory of relativity, "
            "which includes E=mc², time dilation, and the equivalence principle."
        )
    }
]

show_messages(messages)
response_text = chat(messages)

# Try to parse the JSON response
try:
    parsed = json.loads(response_text)
    print("\n✅ Valid JSON! Parsed result:")
    print(json.dumps(parsed, indent=2))
except json.JSONDecodeError:
    print("\n⚠️ Response was not valid JSON — this shows the challenge of format control!")

```json
{
  "name": "Albert Einstein",
  "topic": "Theory of Relativity",
  "key_points": [
    "E=mc² - Mass-energy equivalence formula",
    "Time dilation - Explanation for differences in perceived time between moving observers"
  ]
}
```


⏱️ Response time: 4.38s

⚠️ Response was not valid JSON — this shows the challenge of format control!


---

## 2. The User Prompt — Driving the Conversation

The **user prompt** is the direct input from the human. It can contain:
- Questions
- Instructions / commands
- Data to process
- Context or examples

### Experiment 2A: Vague vs. Specific User Prompts

In [8]:
system = {"role": "system", "content": "You are a helpful assistant. Keep responses concise."}

# --- Vague prompt ---
print("=" * 60)
print("VAGUE USER PROMPT")
print("=" * 60)

vague = [
    system,
    {"role": "user", "content": "Tell me about Python."}
]
show_messages(vague)
_ = chat(vague)

VAGUE USER PROMPT


Python is a high-level, interpreted programming language known for its readability and ease of use. Here are some key points about it:

- It was created by Guido van Rossum in the late 1980s.
- Python uses indentation to define code blocks instead of curly braces or keywords like "if".
- It's open-source software with a large, active community.
- Python is widely used for web development (like Django and Flask), data analysis, artificial intelligence, automation scripts, educational purposes, scientific computing, and more.

Some notable features include:
1. Easy to read/write
2. Dynamic typing system (easier than statically typed languages like C or Java)
3. Built-in support for various modules for tasks such as networking, graphics, and databases

Python is highly versatile and has a wide range of applications across different industries including finance, web development, scientific research, and more.


⏱️ Response time: 2.98s


In [9]:
# --- Specific prompt ---
print("=" * 60)
print("SPECIFIC USER PROMPT")
print("=" * 60)

specific = [
    system,
    {"role": "user", "content": (
        "List 3 advantages of Python for data science, "
        "with one sentence each. Format as a numbered list."
    )}
]
show_messages(specific)
_ = chat(specific)

SPECIFIC USER PROMPT


1. Easy to learn syntax: Python has an intuitive and user-friendly syntax that makes it accessible even for beginners.
2. Large library ecosystem: Python boasts a vast collection of libraries and frameworks specifically designed for scientific computing, making it ideal for data science projects.
3. Strong community support: The Python community is highly supportive, offering extensive resources such as tutorials, forums, and online communities to help users solve problems effectively.


⏱️ Response time: 4.28s


### Experiment 2B: User Prompt with Embedded Data

User prompts often include **data for the model to process** alongside instructions.

In [10]:
messages = [
    {"role": "system", "content": "You are a sentiment analysis assistant. Classify each review and explain briefly."},
    {"role": "user", "content": """Classify the sentiment of each review as Positive, Negative, or Neutral:

Review 1: "This product is amazing! Best purchase I've ever made."
Review 2: "Terrible quality. Broke after one day. Never buying again."
Review 3: "It's okay. Does what it says, nothing special."
"""}
]

show_messages(messages)
_ = chat(messages)

Review 1: Positive  
Review 2: Negative  
Review 3: Neutral


⏱️ Response time: 3.05s


---

## 3. The Assistant Role — Pre-filling & Multi-turn Conversations

The **assistant** role represents the model's own responses. You can use it in two powerful ways:

1. **Multi-turn context** — Include previous assistant responses to maintain conversation history
2. **Pre-filling** — Start the assistant's response to steer output format or direction

### Experiment 3A: Multi-turn Conversation

In [11]:
# Simulating a multi-turn conversation
messages = [
    {"role": "system", "content": "You are a helpful math tutor. Be encouraging and explain step by step."},
    {"role": "user", "content": "What is 15% of 200?"},
    {"role": "assistant", "content": "15% of 200 is 30. Here's how: 15/100 × 200 = 0.15 × 200 = 30."},
    {"role": "user", "content": "Now what if I want to find 15% of 350?"}
]

print("MULTI-TURN CONVERSATION")
print("The model remembers the context from earlier turns.")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

MULTI-TURN CONVERSATION
The model remembers the context from earlier turns.


To find 15% of 350, you can follow these steps:

1. **Understand the percentage**: 15% means that you are looking for 15 parts out of every 100 parts.

2. **Set up a proportion**:
   \[
   \frac{15}{100} = \frac{x}{350}
   \]
   where \( x \) is the amount we're trying to find.

3. **Solve for \( x \)**:
   - Cross-multiply to solve: \( 15 \times 350 = 100 \times x \).
   - Simplify: \( 5250 = 100x \).

4. **Divide both sides by 100**:
   \[
   x = \frac{5250}{100} = 52.5
   \]

So, 15% of 350 is 52.5.


⏱️ Response time: 5.11s


### Experiment 3B: Building a Conversation Turn by Turn

Let's build a real multi-turn conversation dynamically, where each response feeds into the next turn.

In [12]:
conversation = [
    {"role": "system", "content": "You are a travel guide. Give brief, enthusiastic recommendations. Keep each response to 2-3 sentences."}
]

user_turns = [
    "I'm planning a trip to Japan. What should I not miss?",
    "That sounds great! What about food — what must I try?",
    "Any tips for a first-time visitor?"
]

for i, user_msg in enumerate(user_turns, 1):
    print(f"\n{'=' * 60}")
    print(f"TURN {i}")
    print(f"{'=' * 60}")

    conversation.append({"role": "user", "content": user_msg})
    show_messages(conversation)

    response = llm.invoke(build_messages(conversation))
    assistant_msg = response.content
    display(Markdown(assistant_msg))

    # Add the assistant's response to conversation history
    conversation.append({"role": "assistant", "content": assistant_msg})

print(f"\n{'=' * 60}")
print(f"Total messages in conversation: {len(conversation)}")


TURN 1


Japan is a place of incredible contrasts and charm, so don't miss the chance to visit traditional Japanese culture centers like Kyoto or Nara, experiencing temples, shrines, and calligraphy. The cherry blossom season in spring adds an extra layer of beauty and romance with blooming pink trees.

Another must-visit spot is Tokyo's bustling Shibuya district for a taste of modern Japan, from neon lights to high-tech shopping. And don't forget the unique experience at Tsukiji Fish Market, where you can witness the daily auction of sushi-grade fish.

Lastly, immerse yourself in the rich history and artistry by visiting Himeji Castle, known as the 'Castle on a Cloud' for its striking red exterior, offering panoramic views of the surrounding hills.


TURN 2


In Japan, don't miss out on trying authentic sushi at a local restaurant like Sushi Yosai in Kappo or at Tsukiji Market for freshly caught seafood. For something truly special, the Okonomiyaki, a savory pancake filled with various ingredients including cabbage and meat, is highly recommended.

For dessert, try Amanokami, a unique dish consisting of ice cream, jelly (a Japanese candy), and chocolate that's served in a small paper cone. For some unexpected flavors, consider trying kaisendon – cold white fish with hot miso sauce and rice topped on top of ice.


TURN 3


For first-time visitors to Japan, it’s often helpful to understand Japanese characters (kanji) and basic greetings like "Konnichiwa" (good afternoon). Learning some phrases in Japanese can also be beneficial, such as asking directions or ordering food.

The best time to visit Japan is spring when cherry blossoms bloom, but remember that many tourist attractions close during winter. Always check opening times before visiting parks, museums, and other popular sites.

Lastly, respect the local culture by learning about traditional customs like bowing gestures (sotto) as a sign of respect and using chopsticks politely. This small consideration can go a long way in gaining deeper cultural insights into Japan.


Total messages in conversation: 7


---

## 4. How the Three Roles Work Together

The real power comes from combining all three roles effectively.

```
┌─────────────────────────────────────────────────┐
│  SYSTEM: "You are a concise code reviewer."     │  ← Sets behavior
├─────────────────────────────────────────────────┤
│  USER: "Review this function..."                │  ← Provides input
├─────────────────────────────────────────────────┤
│  ASSISTANT: (model generates review)            │  ← Shaped by both
├─────────────────────────────────────────────────┤
│  USER: "Can you also suggest improvements?"     │  ← Follow-up
├─────────────────────────────────────────────────┤
│  ASSISTANT: (uses full context above)           │  ← Informed by all
└─────────────────────────────────────────────────┘
```

### Experiment 4A: Full Pipeline — Code Review Assistant

In [13]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a senior Python code reviewer. "
            "For each review, provide: "
            "1) A brief summary, "
            "2) Issues found (if any), "
            "3) Suggested improvements. "
            "Be constructive and concise."
        )
    },
    {
        "role": "user",
        "content": """Review this Python function:

def calc(x, y, z):
    result = x + y
    result = result * z
    if result > 100:
        return "big"
    else:
        return "small"
"""
    }
]

print("CODE REVIEW ASSISTANT")
print("=" * 60)
show_messages(messages)
_ = chat(messages)

CODE REVIEW ASSISTANT


### Summary
The provided `calc` function is designed to take two inputs (`x`, `y`, and `z`) and multiply their sum by the third input. The function returns either `"big"` or `"small"`. This behavior depends solely on whether the result of multiplying `x + y` by `z` exceeds 100.

### Issues Found
1. **Redundant Operation**: There is an unnecessary operation that occurs both in the calculation and in the conditional block.
2. **Magic Number**: The constant value `100` should be replaced with a defined variable for better readability.
3. **Potential Side Effects**: If any part of this function could potentially mutate input values (e.g., through global variables), it's important to understand what happens if the function is called multiple times.

### Suggested Improvements
1. **Remove Redundant Calculation**:
   - Remove unnecessary computation in the return statement.
   
2. **Define a Variable for 100**:
   - Use `MAX_VALUE` as a named constant, making the intent clearer and more maintainable.

3. **Consider Global Variables** (Optional):
   - If this function is part of a larger application where it might mutate input values, consider defining constants in a separate file or using a different method to manage such variables.
   
Here’s how you can refactor the code:

```python
MAX_VALUE = 100

def calc(x, y, z):
    result = x + y
    if result * z > MAX_VALUE:
        return "big"
    else:
        return "small"

# Example usage:
print(calc(5, 3, 4))  # Should output: small
```

### Explanation of Changes
1. **Redundant Operation Removed**:
   - The line `result = result * z` is now unnecessary because the function already has a valid value (`x + y`) to multiply by.

2. **Magic Number Replaced with Constant**:
   - Changed the constant value from 100 to MAX_VALUE, making it clear what 100 represents and improving readability.

3. **Optional Global Variable Handling (Not Required in This Case)**:
   - If you decide that this function should operate globally or if any part of it might mutate inputs, defining constants can be useful for maintaining clean code and avoiding side effects.


⏱️ Response time: 8.54s


### Experiment 4B: Comparing System Prompt Strictness

How does the **level of detail** in a system prompt affect the model's behavior?

In [14]:
user_msg = "What are the benefits of exercise?"

system_levels = [
    {
        "label": "Minimal",
        "prompt": "You are a helpful assistant."
    },
    {
        "label": "Moderate",
        "prompt": "You are a health expert. Give practical advice in bullet points. Limit to 5 bullets."
    },
    {
        "label": "Strict",
        "prompt": (
            "You are a certified fitness coach. "
            "Rules: "
            "1) Always respond with exactly 3 bullet points. "
            "2) Each bullet must be one sentence only. "
            "3) Include a specific scientific benefit in each. "
            "4) End with a motivational one-liner."
        )
    }
]

for level in system_levels:
    print(f"\n{'=' * 60}")
    print(f"SYSTEM PROMPT LEVEL: {level['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": level["prompt"]},
        {"role": "user", "content": user_msg}
    ]
    show_messages(messages)
    _ = chat(messages)


SYSTEM PROMPT LEVEL: Minimal


The benefits of exercise include improved physical health, mental well-being, weight management, increased energy levels, and reduced risk of chronic diseases. Exercise can also improve overall quality of life by reducing stress, enhancing mood, and promoting better sleep. Additionally, regular physical activity has been shown to reduce the risk of developing certain types of cancer, including breast, colon, and lung cancer.


⏱️ Response time: 1.34s

SYSTEM PROMPT LEVEL: Moderate


1. Boosts Physical Health: Exercise strengthens your heart, improves lung function, enhances muscle strength and endurance, maintains a healthy weight, and lowers blood pressure and cholesterol levels.
2. Improves Mental Health: Regular physical activity has been shown to improve mood, reduce symptoms of depression and anxiety, enhance cognitive function, and increase self-esteem and social connections.
3. Promotes Better Sleep: Exercise can help you fall asleep faster and enjoy deeper sleep, which in turn leads to better overall health and reduces the need for medication.
4. Enhances Physical Appearance: Regular physical activity helps maintain a healthy body weight, improves muscle tone, enhances skin texture, and increases energy levels, leading to a healthier appearance.
5. Increases Energy Levels: Exercise can help improve your stamina and endurance, giving you more energy throughout the day, which in turn reduces fatigue and boosts productivity.


⏱️ Response time: 8.18s

SYSTEM PROMPT LEVEL: Strict


- Exercise enhances heart health, reducing the risk of heart disease.
- It boosts muscle strength and endurance, improving physical performance.
- Increased flexibility reduces body stiffness and pain.


⏱️ Response time: 10.17s


---

## Key Takeaways

| Concept | What to Remember |
|---------|------------------|
| **System Prompt** | Sets the "personality" and rules — processed first, shapes all responses |
| **User Prompt** | Your direct input — be specific for better results |
| **Assistant Role** | Model responses that form conversation memory |
| **Multi-turn** | Including prior assistant messages gives the model conversation context |
| **Specificity** | More detailed system prompts = more controlled outputs |
| **Conflicts** | System prompts generally take priority, but results vary by model |
| **Guardrails** | Use system prompts to constrain the model's scope and behavior |